In [1]:
import torch
import numpy as np
import torch.nn as nn
import torch.functional as F
import matplotlib.pyplot as plt
from torch.utils.data import Dataset, DataLoader
import torchvision
from torchvision import datasets, models, transforms

In [14]:
transform=transforms.Compose([transforms.RandomResizedCrop(24),transforms.ToTensor()])

train = datasets.FashionMNIST(root="FashionMNIST/processed/",train=True,download=True,transform=transform)
test=datasets.FashionMNIST(root="FashionMNIST/processed/",train=False,download=True,transform=transform)

In [28]:
batch_size=32
train_data=DataLoader(train,batch_size=batch_size,drop_last=True,shuffle=True)
test_data=DataLoader(test,batch_size=batch_size,drop_last=True,shuffle=True)

In [20]:
in_features=next(iter(train_data))[0].view(32,-1).shape[-1]
in_features

576

In [23]:
class Net(nn.Module):
    def __init__(self,in_features):
        super().__init__()
        self.fc1=nn.Linear(in_features,128)
        self.fc2=nn.Linear(128,32)
        self.fc3=nn.Linear(32,10)
        self.relu=nn.ReLU()
        self.softmax=nn.Softmax()
        self.dropout=nn.Dropout(0.2)
    
    def forward(self,x):
        x=self.relu(self.fc1(x))
        x=self.dropout(x)
        x=self.relu(self.fc2(x))
        x=self.dropout(x)
        x=self.softmax(self.fc3(x))
        
        return x

In [25]:
model=Net(in_features)
criterion=nn.CrossEntropyLoss()
optimizer=torch.optim.Adam(model.parameters(),lr=0.001)

In [31]:
device=('cuda' if torch.cuda.is_available() else 'cpu')

train_loss_epoch=[]
val_loss_epoch=[]

best_weight=dict()

epoch=10

model.to(device)
criterion.to(device)

for i in range(epoch):
    train_loss=0
    val_loss=0
    
    model.train()
    for images,labels in train_data:
        
        optimizer.zero_grad()
        
        images=images.view(32,-1) # flatten image in input layer
        
#         images=images.type(torch.float)
#         labels=labels.type(torch.float)
        
        images.to(device)
        labels.to(device)
        
        logits=model(images)  #model prediction
        
        loss=criterion(logits,labels) #calculating loss batch wise
        
        
        loss.backward()  #applying the backward propogation
        
        optimizer.step()  #gradient descent
        
        train_loss+=loss  # epoch loss
    train_loss_epoch.append(train_loss)    
    
    #Evaluation phase
    model.eval()
    total_correct=[]
    size=len(test_data.dataset)
    
    with torch.no_grad(): #context manager why because we wont apply gradients in validation phase
        no_of_batches=size//batch_size
        
        for images,labels in test_data:
            images=images.view(32,-1)
            
#             images=images.type(torch.float)
#             labels=labels.type(torch.float)
        
            images.to(device)
            labels.to(device)
            
            logits=model(images)
            val_loss=criterion(logits,labels)
            
            total_val_loss+=val_loss
            
            log_max=logits.max(logits,1).indices
            
    print(f"Running the epoch {i}/{epoch}")
    print("/n")
    print("--"*20)
    print(f'Training loss :{train_loss}')
    print(f'Validation loss :{val_loss}')
    print()
            
            

<ipython-input-23-aaf2543dcdb7>:16: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
  x=self.softmax(self.fc3(x))


KeyboardInterrupt: 